極性分析に有用そうな素性を各自で設計し，学習データから素性を抽出せよ．素性としては，レビューからストップワードを除去し，各単語をステミング処理したものが最低限のベースラインとなるであろう．

In [1]:
import warnings
import re
import csv

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer as PS
import stanfordnlp

In [2]:
# 速くするためにタプルとして定義
STOP_WORDS = set(stopwords.words('english'))

ps = PS()

In [3]:
# Universal POS tags に準拠していそう
# https://universaldependencies.org/u/pos/
EXC_POS = {'PUNCT',   # 句読点
           'X',       # その他
           'SYM',     # 記号
           'PART',    # 助詞('sなど)
           'CCONJ',   # 接続詞(andなど)
           'AUX',     # 助動詞(wouldなど)
           'PRON',    # 代名詞
           'SCONJ',   # 従位接続詞(whetherなど)
           'ADP',     # 接置詞(inなど)
           'NUM'}     # 番号

In [4]:
# プロセッサをデフォルトの全指定にするおｔ遅かったので最低限に絞る
# https://stanfordnlp.github.io/stanfordnlp/processors.html
nlp = stanfordnlp.Pipeline(processors='tokenize,pos,lemma')

Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': '/home/i348221/stanfordnlp_resources/en_ewt_models/en_ewt_tokenizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': '/home/i348221/stanfordnlp_resources/en_ewt_models/en_ewt_tagger.pt', 'pretrain_path': '/home/i348221/stanfordnlp_resources/en_ewt_models/en_ewt.pretrain.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': '/home/i348221/stanfordnlp_resources/en_ewt_models/en_ewt_lemmatizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
Done loading processors!
---


In [5]:
reg_sym = re.compile(r'^[!-/:-@[-`{-~]|[!-/:-@[-`{-~]$')
reg_dit = re.compile('[0-9]')

In [6]:
# 先頭と末尾の記号除去
def remove_symbols(lemma):
    return ps.stem(reg_sym.sub('', lemma))

In [7]:
# ストップワード真偽判定
def is_stopword(word):
    lemma = remove_symbols(word.lemma)
    return True if lemma in STOP_WORDS \
                  or lemma == '' \
                  or word.upos in EXC_POS \
                  or len(lemma) == 1 \
                  or reg_dit.search(lemma)\
                else False

In [8]:
# 警告非表示
warnings.simplefilter('ignore', UserWarning)

In [9]:
%%time
with open('./sentiment.txt') as file_in:
    with open('./sentiment_stem.txt', 'w') as file_out:
        writer = csv.writer(file_out, delimiter='\t')
        writer.writerow(['Lable', 'Lemmas'])

        for i, line in enumerate(file_in):
            print("\r{0}".format(i), end="")
        
            lemma = []
        
            # 最初の3文字はネガポジを示すだけなのでnlp処理しない(少しでも速くする)
            doc = nlp(line[3:])
            for sentence in doc.sentences:
                lemma.extend([remove_symbols(word.lemma) for word in sentence.words if is_stopword(word) is False])
            writer.writerow([1 if line[0] == '+' else 0, ' '.join(lemma)])
            
#            if i == 10:
#                break

10661CPU times: user 1h 27min 7s, sys: 4min 4s, total: 1h 31min 11s
Wall time: 39min 19s
